In [ ]:
from bs4 import BeautifulSoup 
import requests 

import re

import time
import random

В ДАННОМ НОУТБУКЕ В УДОБОЧИТАЕМОМ ВИДЕ ПОКАЗАН ПРОЦЕСС ПОЛУЧЕНИЯ ДАННЫХ С САЙТА. КОД ЗАПУСКАЛСЯ ИЗ ДРУГИХ НОУТБУКОВ, КОТОРЫЕ НАХОДЯТСЯ В ПАПКЕ DRAFTS

# ЗАДАЕМ ФУНКЦИЮ ДЛЯ ИЗВЛЕЧЕНИЯ ЦИТАТ И ОТВЕТ НА ЦИТАТЫ НА САЙТЕ https://forum.say7.info/cat14.html

In [ ]:
def extract_dialogue(url, debug = False):
    page = requests.get(url)
    page_source = page.content
    soup = BeautifulSoup(page_source)
    #if debug == True: print(soup)
    dialogues = []
    curr_dial = []
    add_response = False
    for line in soup.get_text().split("\n"):
        if debug == True: print(line)
        cleaned_line = re.sub(' +', ' ', line)
        cleaned_line = cleaned_line.strip()


        if(add_response and cleaned_line):
            if debug == True: print("REPLY DETECTED")
            curr_dial.append(cleaned_line)
            add_response = False

            assert (len(curr_dial) == 2)
            dialogues.append(curr_dial)
            curr_dial = []

        if "писал(а):" in cleaned_line:
            start_quote_info = cleaned_line.index("писал(а):")
            if debug == True: print("FIRST PHRASE DETECTED")
            add_response = True
            if debug == True: print(cleaned_line[start_quote_info + 10:])
            curr_dial.append(cleaned_line[start_quote_info + 10:])
        if debug == True: print("==========")
    return dialogues
    
lk = 'https://forum.say7.info/topic95678.html'
extract_dialogue(lk, debug = False)

# ПРЕДВАРИТЕЛЬНО СОБИРАЕМ ССЫЛКИ НА ОБСУЖДЕНИЯ РЕЦЕПТОВ С ПЕРВЫХ 50-ТИ СТРАНИЦ КАЖДОЙ ТЕМЫ

In [ ]:
def get_recipies_links(forum_link, page_range):
    recipies_links = []
    fisrt_page = [None]
    pages = [25 * i for i in range(1, page_range)]
    all_pages_ind = fisrt_page + pages
    
    substring_index = forum_link.index(".html")
    
    for ind in all_pages_ind:
        time.sleep(10 + random.randint(1,10))
        
        if(ind):
            forum_link = forum_link[:substring_index] + "-" + str(ind) + ".html"
        else:
            forum_link = forum_link

        #print(forum_link)

        r = requests.get(forum_link)

        forum_index =forum_link.split("/")[-1]
        forum_list = BeautifulSoup(r.content)
        for link in forum_list.find_all("a", href = True):
            if link.text: 
                lk = link['href']
                if("topic" in lk and "topics" not in lk):
                    updated_link = forum_link.replace(forum_index, lk)
                    #print("LINK", updated_link)
                    recipies_links.append(updated_link)
    return recipies_links

In [ ]:
topics_links_basic = get_recipies_links("https://forum.say7.info/forum12.html", 2 )
topics_links_vypechka = get_recipies_links("https://forum.say7.info/forum68.html", 50)
topics_links_first = get_recipies_links("https://forum.say7.info/forum64.html", 50)
topics_links_second = get_recipies_links("https://forum.say7.info/forum65.html", 50)
topics_links_fish = get_recipies_links("https://forum.say7.info/forum74.html", 50)
topics_links_desert = get_recipies_links("https://forum.say7.info/forum69.html", 50)
topics_links_porridge = get_recipies_links("https://forum.say7.info/forum70.html", 50)

In [ ]:
def get_dialogues_from_links_list(links_list):
    bar = progressbar.ProgressBar(maxval=len(links_list),
                                  widgets=[progressbar.Bar('=', '[', ']'), ' ', 
                                           progressbar.Percentage()])
    bar.start()
    lines = 0

    dialogues = []
    for link in links_list:
        time.sleep(10 + random.randint(1,10))
        curr_dialogues = extract_dialogue(link)
        if(lines%10 == 0): print(curr_dialogues)
        dialogues.extend(curr_dialogues)
        
        lines += 1
        bar.update(lines)
        sleep(0.1) 
    return dialogues

In [ ]:
basic = get_recipies_links(topics_links_basic)
vypechka = get_recipies_links(topics_links_vypechka)
first = get_recipies_links(topics_links_first)
second = get_recipies_links(topics_links_second)
fish = get_recipies_links(topics_links_fish)
desert = get_recipies_links(topics_links_desert)
porridge = get_recipies_links(topics_links_porridge)

In [ ]:
def save_dialogues(dialogues, df_name):
    fist_phrase = [phrase[0] for phrase in dialogues]
    reply = [phrase[1] for phrase in dialogues]
    dialogues_df = pd.DataFrame(
    {'fist_phrase': fist_phrase,
     'reply': reply,
    })
    df_name = "second.csv"
    dialogues_df.to_csv(df_name, sep='\t', encoding='utf-8', index=False)

In [ ]:
save_dialogues(basic,"basic.csv")
save_dialogues(vypechka,"vypechka")
save_dialogues(first,"first.csv")
save_dialogues(second,"second.csv")
save_dialogues(fish,"fish.csv")
save_dialogues(desert,"desert.csv")
save_dialogues(porridge,"porridge.csv")